In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.extend([ '../', '../../'])
import torch
import torch.nn as nn
import torch.optim as optim
from data import input_data
import dataloader as dl
from args import Args
from model import Het_En, Classifier, EdgePredictor, Het_classify
from smote import oversample
from train import train_smote, test_smote
import cProfile

# Set device to GPU if available, else use CPU
args = Args()
args.aminer_train()
torch.cuda.empty_cache()

Current device: NVIDIA A100 80GB PCIe


In [2]:
# data = input_data(args) 
# torch.save(data, 'data.pt')
data = torch.load('../../data/aminer/am_data.pt')
print(data)

HeteroData(
  a={
    num_nodes=20171,
    y=[20171]
  },
  p={ num_nodes=13250 },
  v={ num_nodes=18 },
  p_title_embed={ x=[13250, 128] },
  p_abstract_embed={ x=[13250, 128] },
  p_net_embed={ x=[13250, 128] },
  p_a_net_embed={ x=[13250, 128] },
  p_p_net_embed={ x=[13250, 128] },
  p_v_net_embed={ x=[13250, 128] },
  a_net_embed={ x=[20171, 128] },
  a_text_embed={ x=[20171, 128] },
  v_net_embed={ x=[18, 128] },
  v_text_embed={ x=[18, 128] },
  (a, walk, a)={ edge_index=[2, 99480] },
  (a, walk, p)={ edge_index=[2, 86444] },
  (a, walk, v)={ edge_index=[2, 31716] },
  (p, walk, a)={ edge_index=[2, 65534] },
  (p, walk, p)={ edge_index=[2, 66174] },
  (p, walk, v)={ edge_index=[2, 20838] },
  (v, walk, a)={ edge_index=[2, 90] },
  (v, walk, p)={ edge_index=[2, 90] },
  (v, walk, v)={ edge_index=[2, 27] }
)


In [3]:
print(data['a','walk','p'].edge_index)
device = args.device
# data = data.to(device)

# Send all x tensors to the device
data['p_title_embed']['x'] = data['p_title_embed']['x'].to(device)
data['p_abstract_embed']['x'] = data['p_abstract_embed']['x'].to(device)
data['p_net_embed']['x'] = data['p_net_embed']['x'].to(device)
data['p_a_net_embed']['x'] = data['p_a_net_embed']['x'].to(device)
data['p_p_net_embed']['x'] = data['p_p_net_embed']['x'].to(device)
data['p_v_net_embed']['x'] = data['p_v_net_embed']['x'].to(device)
data['a_net_embed']['x'] = data['a_net_embed']['x'].to(device)
data['a_text_embed']['x'] = data['a_text_embed']['x'].to(device)
data['v_net_embed']['x'] = data['v_net_embed']['x'].to(device)
data['v_text_embed']['x'] = data['v_text_embed']['x'].to(device)

# Send all y tensors to the device
data['a']['y'] = data['a']['y'].to(device)

# Send all edge_index tensors to the device
data['a', 'walk', 'a']['edge_index'] = data['a', 'walk', 'a']['edge_index'].to(device)
data['a', 'walk', 'p']['edge_index'] = data['a', 'walk', 'p']['edge_index'].to(device)
data['a', 'walk', 'v']['edge_index'] = data['a', 'walk', 'v']['edge_index'].to(device)
data['p', 'walk', 'a']['edge_index'] = data['p', 'walk', 'a']['edge_index'].to(device)
data['p', 'walk', 'p']['edge_index'] = data['p', 'walk', 'p']['edge_index'].to(device)
data['p', 'walk', 'v']['edge_index'] = data['p', 'walk', 'v']['edge_index'].to(device)
data['v', 'walk', 'a']['edge_index'] = data['v', 'walk', 'a']['edge_index'].to(device)
data['v', 'walk', 'p']['edge_index'] = data['v', 'walk', 'p']['edge_index'].to(device)
data['v', 'walk', 'v']['edge_index'] = data['v', 'walk', 'v']['edge_index'].to(device)

edge_indices = [ data['a', 'walk', 'a'].edge_index, data['a', 'walk', 'p'].edge_index, data['a', 'walk', 'v'].edge_index ]

tensor([[    0,     0,     0,  ..., 20170, 20170, 20170],
        [10523, 10442, 10329,  ...,  4378, 12456,  6612]])


In [4]:
c_train_num = dl.train_num(data['a'].y, args.im_class_num, args.class_samp_num[0], args.im_ratio)
print(c_train_num, sum(c_train_num))
train_idx, val_idx, test_idx, c_num_mat = dl.segregate(data['a'].y, c_train_num, args)
print("train_idx: ", train_idx, len(train_idx))
print("val_idx: ", val_idx, len(val_idx))
print("test_idx: ", test_idx, len(test_idx))
# print(c_num_mat)

[50, 30, 25, 20] 125
0 335
1 202
2 299
3 289
train_idx:  [607, 15697, 11409, 1398, 13584, 12239, 13137, 1361, 15057, 8072, 5352, 14436, 14547, 17783, 19784, 215, 4870, 16768, 18004, 160, 11728, 17328, 7858, 4393, 4945, 3627, 9739, 17924, 2252, 1119, 13039, 17692, 18849, 19229, 8398, 895, 8058, 8721, 3596, 19066, 2985, 11482, 4242, 4534, 3396, 14674, 17095, 9519, 16001, 16232, 19291, 16326, 12250, 12263, 9709, 3540, 4003, 8410, 7549, 3571, 11992, 16337, 9795, 7309, 1977, 13302, 10399, 19234, 11915, 11823, 11659, 16458, 8942, 7104, 1268, 4658, 4325, 10352, 1943, 14221, 13507, 19663, 1892, 15679, 630, 15291, 407, 7308, 6433, 8272, 4095, 6833, 4907, 7277, 2918, 19121, 18983, 6663, 18642, 19186, 550, 4844, 15243, 19102, 1126, 5675, 13942, 7441, 18214, 10741, 59, 1149, 4578, 1960, 16803, 15651, 6950, 12681, 8059, 3413, 16279, 15984, 7125, 7953, 9688] 125
val_idx:  [19506, 151, 4461, 3114, 7069, 19661, 17533, 18889, 3053, 13132, 6076, 10885, 1538, 2296, 10984, 15168, 12522, 9855, 2887, 5739, 

In [5]:
# os_mode = 'up'
# portion = sum(c_train_num)/(4*c_train_num[1])
# if os_mode != 'reweight' and os_mode != 'no':
#     if portion >= 1:
#         args.node_dim[0] = args.A_n + (portion-1)*c_train_num[1] + int((portion - int(portion))*c_train_num[1])
#     else:
#         args.node_dim[0] = args.A_n + int((portion - int(portion))*c_train_num[1])

# args.node_dim[0] += 22

In [6]:
encoder = Het_En(args.embed_dim, args.dropout)
classifier = Het_classify(args.embed_dim, args.nclass, args.dropout)
decoder_a = EdgePredictor(args.embed_dim)
decoder_p = EdgePredictor(args.embed_dim)
decoder_v = EdgePredictor(args.embed_dim)
decoder_list = [decoder_a, decoder_p, decoder_v]
#print(features.shape)
encoder.to(device)
classifier.to(device)
for decoder in decoder_list:
    decoder.to(device)

## Training Part

In [7]:
torch.cuda.empty_cache()
# profiler = cProfile.Profile()
# profiler.enable()
train_smote(data, edge_indices, encoder, classifier, decoder_list, train_idx, val_idx, test_idx, args, os_mode = 'gsm', train_mode = 'preo')
# profiler.disable()

Epoch [1/200], Loss: 1.4106, Accuracy: 0.2063, Edge Accuracy: [0.9999950795002466, 0.9999972948273395, 0.9128396653060775]
Class 0: AUC-ROC- 0.6016, F1 Score- 0.0000; Class 1: AUC-ROC- 0.4034, F1 Score- 0.2472; Class 2: AUC-ROC- 0.6844, F1 Score- 0.0000; Class 3: AUC-ROC- 0.4288, F1 Score- 0.0000; Macro-Average AUC-ROC: 0.5295; Macro-Average F1 Score: 0.0618
Validation Loss: 1.3861, Validation Accuracy: 0.2591, Validation Edge Accuracy: []
Class 0: AUC-ROC- 0.6417, F1 Score- 0.0000; Class 1: AUC-ROC- 0.2437, F1 Score- 0.1648; Class 2: AUC-ROC- 0.6786, F1 Score- 0.0000; Class 3: AUC-ROC- 0.5671, F1 Score- 0.0000; Macro-Average AUC-ROC: 0.5328; Macro-Average F1 Score: 0.0412
Test Loss: 1.3866, Test Accuracy: 0.2600, Test Edge Accuracy: []
Class 0: AUC-ROC- 0.6721, F1 Score- 0.0000; Class 1: AUC-ROC- 0.3161, F1 Score- 0.2348; Class 2: AUC-ROC- 0.6260, F1 Score- 0.0000; Class 3: AUC-ROC- 0.4948, F1 Score- 0.0000; Macro-Average AUC-ROC: 0.5272; Macro-Average F1 Score: 0.0587

Epoch [11/200]

In [8]:
# profiler.print_stats(sort='cumulative')  # Print results to the console
# # Or save to a file
# with open('profile_results.txt', 'w') as f:
#     profiler.dump_stats(f)

In [9]:
test_smote(data, edge_indices, encoder, classifier, decoder_list, test_idx, args = args, dataset = 'Test', os_mode = 'no', train_mode = 'preo')

Test Loss: 1.0090, Test Accuracy: 0.9667, Test Edge Accuracy: []
Class 0: AUC-ROC- 0.9998, F1 Score- 0.9740; Class 1: AUC-ROC- 0.9999, F1 Score- 0.8982; Class 2: AUC-ROC- 0.9964, F1 Score- 0.9299; Class 3: AUC-ROC- 0.9998, F1 Score- 0.9726; Macro-Average AUC-ROC: 0.9989; Macro-Average F1 Score: 0.9437


(0.9666666666666667,
 0.9989481481481481,
 0.9436921530747142,
 [0.999762962962963,
  0.9998814814814815,
  0.9963851851851853,
  0.999762962962963],
 [0.974025974025974,
  0.8982035928143712,
  0.9299363057324841,
  0.9726027397260273])

In [10]:
# torch.save(decoder_list[0], 'pre_decoder/decoder_a.pth')
# torch.save(decoder_list[1], 'pre_decoder/decoder_p.pth')
# torch.save(decoder_list[2], 'pre_decoder/decoder_v.pth')
torch.save(encoder, 'pre_decoder/encoder.pth')